In [1]:
%cd ../..

c:\Users\HuyLQ15_CTV\Desktop\model_monitoring


In [2]:
from datetime import time, datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
from datetime import datetime, time
from source.mlops_observation import ColumnMapping

content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip", verify=False).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')
raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-31 00:00:00']
current = raw_data.loc['2011-02-01 00:00:00':'2011-07-28 23:00:00']
current['time_stamp'] = current.index
reference['time_stamp'] = reference.index

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ho-proxy02.tpb.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
# from evidently.model.widget import BaseWidgetInfo
# from evidently.base_metric import MetricResult, Metric, InputData

# from typing import Optional, Dict, List
# from evidently.model.widget import BaseWidgetInfo
# from evidently.base_metric import InputData, Metric, MetricResult
# from evidently import ColumnMapping

# from evidently.renderers.html_widgets import ColumnDefinition
# from evidently.renderers.html_widgets import RichTableDataRow
# from evidently.renderers.html_widgets import RowDetails
# from evidently.renderers.html_widgets import CounterData
# from evidently.renderers.html_widgets import rich_table_data
# from evidently.renderers.html_widgets import header_text
# from evidently.renderers.html_widgets import plotly_figure
# from evidently.renderers.html_widgets import rich_table_data
# from evidently.renderers.html_widgets import table_data
# from evidently.renderers.html_widgets import counter
# from evidently.utils.visualizations import plot_distr
# from evidently.renderers import html_widgets
# from evidently.renderers.html_widgets import plotly_data
# import plotly.graph_objects as go


In [4]:
# from typing_extensions import Self
# import pandas as pd
# class DriftPeriodResult:
#     distribution: go.Figure
#     drift_period: go.Figure
#     col_type: str
#     col_name: str
#     stat_table: pd.DataFrame
#     period_count: int
#     period_drifted_count: int
#     reference_date_list: int
#     current_date_list: int
#     def __init__(self, distribution, drift_period, col_type, col_name, stat_table, period_drifted_count, period_count):
#         self.distribution = distribution
#         self.drift_period = drift_period
#         self.col_type = col_type
#         self.col_name = col_name
#         self.stat_table = stat_table
#         self.period_count = period_count
#         self.period_drifted_count = period_drifted_count

In [5]:
# from source.mlops_observation.utils.data_drift import convert_test_period_into_dataframe
# from source.mlops_observation.metric_results.data_drift.feature_drift_period import calculate_test_in_period
# from source.mlops_observation.utils.data_drift import convert_test_period_into_dataframe
# from source.mlops_observation.utils.visualization import drift_combine_plot
# from source.mlops_observation.utils.visualization import dataframe_to_widget
# from source.mlops_observation.utils.visualization import period_numerical_distribution_drift
# from source.mlops_observation.utils.visualization import period_categorical_distribution_drift
# from source.mlops_observation.options.color_scheme import ColorOptions
# from source.mlops_observation.renders.base_renders import MetricRenderer
# from source.mlops_observation.utils.data_drift import calculate_period_drifted_dict
# from evidently.renderers.base_renderer import default_renderer

# class PeriodDataDrifts(MetricResult):
#     class Config:
#         type_alias = "evidently:metric_result:DataDriftResult"
#     feature_period_plots: Optional[Dict[str, DriftPeriodResult]]
#     reference_date_list: list
#     current_date_list: list
# class PeriodDataDriftMetric(Metric[PeriodDataDrifts]):
#     """
#     time_period (str): Offset aliases https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
#     """
#     class Config:
#         type_alias = "evidently:metric:DataDriftMetric"
#     _time_period: str
#     def __init__(self, time_period: str = 'M'):
#         self._time_period = time_period
#         super().__init__()
#     def calculate(self, data: InputData):
#         if data.reference_data is None:
#             raise ValueError("Reference dataset should be present")
#         if data.current_data is None:
#             raise ValueError("Current dataset should be present")
#         result = {}
#         column_mapping = data.column_mapping
#         if column_mapping.categorical_features is not None:
#             for cat_col in column_mapping.categorical_features:
#                 distribution_figure = period_categorical_distribution_drift(
#                     reference=data.reference_data,
#                     current=data.current_data,
#                     col_name=cat_col,
#                     timestamp_col=column_mapping.datetime_features,
#                     chunk_period=self._time_period
#                 )
#                 stat_result = calculate_test_in_period(
#                     reference=data.reference_data,
#                     current=data.current_data,
#                     col_name=cat_col,
#                     col_type='cat',
#                     chunk_period=self._time_period,
#                     time_stamp_col=column_mapping.datetime_features,
#                 )
#                 # -----------------------------------
#                 reference_date_list, current_date_list, current_drifted = calculate_period_drifted_dict(
#                     reference=data.reference_data,
#                     timestamp_col=column_mapping.datetime_features,
#                     period_stat_result=stat_result
#                 )
#                 period_drifted_count = sum(current_drifted)
#                 period_count = len(current_date_list)
#                 # -----------------------------------
#                 drift_figure = drift_combine_plot(stat_result)
#                 stat_table = convert_test_period_into_dataframe(stat_result)
#                 result[cat_col] = DriftPeriodResult(
#                     distribution=distribution_figure,
#                     drift_period=drift_figure,
#                     col_type='cat',
#                     col_name=cat_col,
#                     stat_table=stat_table,
#                     period_drifted_count=period_drifted_count,
#                     period_count=period_count
#                 )
#         if column_mapping.numerical_features is not None:
#             for num_col in column_mapping.numerical_features:
#                 distribution_figure = period_numerical_distribution_drift(
#                     reference=data.reference_data,
#                     current=data.current_data,
#                     col_name=num_col,
#                     timestamp_col=column_mapping.datetime_features,
#                     chunk_period=self._time_period
#                 )
#                 stat_result = calculate_test_in_period(
#                     reference=data.reference_data,
#                     current=data.current_data,
#                     col_name=num_col,
#                     col_type='num',
#                     chunk_period=self._time_period,
#                     time_stamp_col=column_mapping.datetime_features,
#                 )
#                 # -------------------------------------
#                 reference_date_list, current_date_list, current_drifted = calculate_period_drifted_dict(
#                     reference=data.reference_data,
#                     timestamp_col=column_mapping.datetime_features,
#                     period_stat_result=stat_result
#                 )
#                 period_drifted_count = sum(current_drifted)
#                 period_count = len(current_date_list)
#                 # -------------------------------------
#                 drift_figure = drift_combine_plot(stat_result)
#                 stat_table = convert_test_period_into_dataframe(stat_result)
#                 result[num_col] = DriftPeriodResult(
#                     distribution=distribution_figure,
#                     drift_period=drift_figure,
#                     col_type='num',
#                     col_name=num_col,
#                     stat_table=stat_table,
#                     period_drifted_count=period_drifted_count,
#                     period_count=period_count
#                 )
#         return PeriodDataDrifts(
#             feature_period_plots=result, 
#             current_date_list=current_date_list, 
#             reference_date_list=reference_date_list
#             )

# @default_renderer(wrap_type=PeriodDataDriftMetric)
# class DataDriftRender(MetricRenderer):
#     def _generate_column_params(
#         self,
#         column_data: DriftPeriodResult
#         )-> Optional[RichTableDataRow]:
#         details = RowDetails()
#         # -----------------------------------------
#         table = column_data.stat_table
#         table = dataframe_to_widget(table)
#         details.with_part("STAT TABLE", info=table)
#         # -----------------------------------------
#         dist_fig = column_data.distribution
#         dist_fig = plotly_figure(title='', figure=dist_fig)
#         details.with_part("DISTRIBUTION", info=dist_fig)
#         # -----------------------------------------
#         drift_fig = column_data.drift_period
#         drift_fig = plotly_figure(title='', figure=drift_fig)
#         details.with_part("DRIFT", info=drift_fig)
#         # -----------------------------------------
        
#         return RichTableDataRow({
#             "column_name": column_data.col_name, 
#             "column_type": column_data.col_type,
#             "period_drifted_count": fr"{column_data.period_drifted_count}/{column_data.period_count}"
#             }, 
#             details=details)
        
#     def render_html(self, obj: PeriodDataDriftMetric) -> List[BaseWidgetInfo]:
#         results = obj.get_result()
#         color_options = self.color_options
#         data=[]
#         counters = [
#             CounterData.string(f"{results.reference_date_list[0]} to {results.reference_date_list[-1]}", 'Reference'),
#             CounterData.string(f"{results.current_date_list[0]} to {results.current_date_list[-1]}", "Current")
#         ]
#         columns = [
#             ColumnDefinition("Column Name", "column_name"),
#             ColumnDefinition("Type", "column_type"),
#             ColumnDefinition("Number of drifted periods", "period_drifted_count")
#         ]
#         for col in results.feature_period_plots.keys():
#             column_data = results.feature_period_plots[col]
#             param = self._generate_column_params(
#                     column_data=column_data
#             )
#             data.append(param)
#         return [
#             header_text(label=f"Period Drift Report"),
#             counter(counters=counters),
#             rich_table_data(
#                 title="",
#                 columns=columns,
#                 data=data
#             )
#         ]

In [6]:
from source.mlops_observation import Report
from source.mlops_observation import ColumnMapping
from source.mlops_observation.metrics import PeriodDataDriftMetric
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features
column_mapping.datetime_features = 'time_stamp'

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.MetricResult'>, 'evidently:metric_result:DatasetUtilityColumns') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.MetricResult'>, 'evidently:metric_result:DatasetColumns') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")


In [ ]:
report = Report([PeriodDataDriftMetric(time_period='M')])
report.run(reference_data=reference, current_data=current, column_mapping=column_mapping)
report.save_html('1.html')